<a href="https://colab.research.google.com/github/HussainAhmed202/my-bioinformatics-journal/blob/main/Projects/Protein_Function_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Neural Network-Based Multi-Label Classifier for Protein Function Prediction

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
!pip install unrar
!unrar x  "/content/drive/MyDrive/Dataset.rar" "/content"



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/Dataset.rar

Creating    /content/Test Dataset                                     OK
Extracting  /content/Test Dataset/Act_test.csv                             0%  1%  2%  OK 
Extracting  /content/Test Dataset/Bac_test.csv                             2%  OK 
Extracting  /content/Test Dataset/Chl_test.csv                             3%  OK 
Extracting  /content/Test Dataset/Cya_test.csv                             4%  OK 
Extracting  /content/Test Dataset/Fir_test1.csv                            5%  6%  7%  8%  OK 
Extracting  /content/Test Dataset/Fir_test2.csv                            9% 10% 11%  OK 
Extracting  /content/Test Dataset/Fuso_test.csv                           11%  OK 
Extracting  /content/Test Dataset/Pro_test1.csv                           12% 13% 14%

In [ ]:
!nvidia-smi

Sun Feb  9 08:09:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Import all Libraries

In [3]:
# import cudf
# %load_ext cudf.pandas
import pandas as pd
import sys


# Import all the datasets

## Actinobacteria

In [49]:
data_Actinobacteria1 = pd.read_csv('/content/Train Dataset/Act_train1.csv')
data_Actinobacteria1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9323), int64(2306), object(3)
memory usage: 440.0 MB


The memory usage for this dataset is **440MB**. We can reduce this by using other int and float datatypes for smaller number

In Pandas for **Integer** data the default dtype is **int64**

* **int64** is a signed 64-bit integer range -(2^63) -> (2^63 - 1)

Other options are **int8** and **int16**
* **int16** is a signed 16-bit integer range -(2^15) -> (2^15 - 1)
* **int8** is a signed 8-bit integer range -(2^7) -> (2^7 - 1)


In [ ]:
# define the ranges
min_val_8bit, max_val_8bit = -2**7, 2**7 - 1
min_val_16bit, max_val_16bit = -2**15, 2**15 - 1

In [50]:
# Goal: Iterate over all integer columns
# If their max and min value can be represented in 8-bit binary then replace with int8
# If their max and min value can be represented in 16-bit binary then replace with int16

int_cols = data_Actinobacteria1.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Actinobacteria1[col].max() < max_val_8bit) and (data_Actinobacteria1[col].min() > min_val_8bit):
    data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('int8')
  elif (data_Actinobacteria1[col].max() < max_val_16bit) and (data_Actinobacteria1[col].min() > min_val_16bit):
      data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('int16')

In [51]:
# check change in size
data_Actinobacteria1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9323), int16(1), int64(2), int8(2303), object(3)
memory usage: 364.2 MB


In Pandas for **Floating Point** data the default dtype is float64. Floating point numbers are stored using **IEEE 754** standard

* **float64** is double precision floating point number ranging -1.7e+308 to +1.7e+308.

Other options are **float16** and **float32**
* **float32** is a single precision range from -3.4e+38 to 3.4e+38.

* **float16** is a half  precision range from -(6.55×104) to +(6.55×104)




In [ ]:
# define the ranges
min_val_16bit, max_val_16bit = -6.55*(10**4), 6.55*(10**4)
min_val_32bit, max_val_32bit = -3.4*(10**38), 3.4*(10**38)

In [52]:
# Goal: Iterate over all float columns
# If their max and min value can be represented in 16-bit binary then replace with float16
# If their max and min value can be represented in 32-bit binary then replace with float32

float_cols = data_Actinobacteria1.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Actinobacteria1[col].max() < max_val_16bit) and (data_Actinobacteria1[col].min() > min_val_16bit):
      data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('float16')
  elif (data_Actinobacteria1[col].max() < max_val_32bit) and (data_Actinobacteria1[col].min() > min_val_32bit):
    data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('float32')


In [53]:
# check change in size
data_Actinobacteria1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9322), float32(1), int16(1), int64(2), int8(2303), object(3)
memory usage: 101.1 MB


The size has been reduced to **101.1MB**

In [55]:
data_Actinobacteria2 = pd.read_csv('/content/Train Dataset/Act_train2.csv')
data_Actinobacteria2.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9323), int64(2306), object(3)
memory usage: 440.1 MB


In [56]:
# reduce any int64 columns
int_cols = data_Actinobacteria2.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Actinobacteria2[col].max() < max_val_8bit) and (data_Actinobacteria2[col].min() > min_val_8bit):
    data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('int8')
  elif (data_Actinobacteria2[col].max() < max_val_16bit) and (data_Actinobacteria2[col].min() > min_val_16bit):
      data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('int16')


# reduce any float64 columns
float_cols = data_Actinobacteria2.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Actinobacteria2[col].max() < max_val_16bit) and (data_Actinobacteria2[col].min() > min_val_16bit):
      data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('float16')
  elif (data_Actinobacteria2[col].max() < max_val_32bit) and (data_Actinobacteria2[col].min() > min_val_32bit):
    data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('float32')



data_Actinobacteria2.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9322), float32(1), int16(1), int64(2), int8(2303), object(3)
memory usage: 101.1 MB


## Bacteroidetes

In [57]:
data_Bacteroidetes = pd.read_csv('/content/Train Dataset/Bac_train.csv')
data_Bacteroidetes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3149 entries, 0 to 3148
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(11629), object(3)
memory usage: 280.9 MB


In [58]:
# reduce any int64 columns
int_cols = data_Bacteroidetes.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Bacteroidetes[col].max() < max_val_8bit) and (data_Bacteroidetes[col].min() > min_val_8bit):
    data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('int8')
  elif (data_Bacteroidetes[col].max() < max_val_16bit) and (data_Bacteroidetes[col].min() > min_val_16bit):
      data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('int16')


# reduce any float64 columns
float_cols = data_Bacteroidetes.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Bacteroidetes[col].max() < max_val_16bit) and (data_Bacteroidetes[col].min() > min_val_16bit):
      data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('float16')
  elif (data_Bacteroidetes[col].max() < max_val_32bit) and (data_Bacteroidetes[col].min() > min_val_32bit):
    data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('float32')



data_Bacteroidetes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3149 entries, 0 to 3148
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(11626), float32(3), object(3)
memory usage: 71.4 MB


## Chlamydiae

In [59]:
data_Chlamydiae = pd.read_csv('/content/Train Dataset/Chl_train.csv')
data_Chlamydiae.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9214), int64(2415), object(3)
memory usage: 116.8 MB


In [60]:
# reduce any int64 columns
int_cols = data_Chlamydiae.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Chlamydiae[col].max() < max_val_8bit) and (data_Chlamydiae[col].min() > min_val_8bit):
    data_Chlamydiae[col] = data_Chlamydiae[col].astype('int8')
  elif (data_Chlamydiae[col].max() < max_val_16bit) and (data_Chlamydiae[col].min() > min_val_16bit):
      data_Chlamydiae[col] = data_Chlamydiae[col].astype('int16')


# reduce any float64 columns
float_cols = data_Chlamydiae.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Chlamydiae[col].max() < max_val_16bit) and (data_Chlamydiae[col].min() > min_val_16bit):
      data_Chlamydiae[col] = data_Chlamydiae[col].astype('float16')
  elif (data_Chlamydiae[col].max() < max_val_32bit) and (data_Chlamydiae[col].min() > min_val_32bit):
    data_Chlamydiae[col] = data_Chlamydiae[col].astype('float32')



data_Chlamydiae.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9213), float32(1), int16(1), int64(2), int8(2412), object(3)
memory usage: 26.7 MB


## Cyanobacteria

In [61]:
data_Cyanobacteria = pd.read_csv('/content/Train Dataset/Cya_train.csv')
data_Cyanobacteria.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5459 entries, 0 to 5458
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9318), int64(2311), object(3)
memory usage: 487.0 MB


In [62]:
# reduce any int64 columns
int_cols = data_Cyanobacteria.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Cyanobacteria[col].max() < max_val_8bit) and (data_Cyanobacteria[col].min() > min_val_8bit):
    data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('int8')
  elif (data_Cyanobacteria[col].max() < max_val_16bit) and (data_Cyanobacteria[col].min() > min_val_16bit):
      data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('int16')


# reduce any float64 columns
float_cols = data_Cyanobacteria.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Cyanobacteria[col].max() < max_val_16bit) and (data_Cyanobacteria[col].min() > min_val_16bit):
      data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('float16')
  elif (data_Cyanobacteria[col].max() < max_val_32bit) and (data_Cyanobacteria[col].min() > min_val_32bit):
    data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('float32')



data_Cyanobacteria.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5459 entries, 0 to 5458
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9317), float32(1), int16(1), int64(2), int8(2308), object(3)
memory usage: 111.8 MB


## Firmicutes

In [63]:
data_Firmicutes1 = pd.read_csv('/content/Train Dataset/Fir_train1.csv')
data_Firmicutes1.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9329), int64(2300), object(3)
memory usage: 240.7 MB


In [64]:
# reduce any int64 columns
int_cols = data_Firmicutes1.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Firmicutes1[col].max() < max_val_8bit) and (data_Firmicutes1[col].min() > min_val_8bit):
    data_Firmicutes1[col] = data_Firmicutes1[col].astype('int8')
  elif (data_Firmicutes1[col].max() < max_val_16bit) and (data_Firmicutes1[col].min() > min_val_16bit):
      data_Firmicutes1[col] = data_Firmicutes1[col].astype('int16')


# reduce any float64 columns
float_cols = data_Firmicutes1.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Firmicutes1[col].max() < max_val_16bit) and (data_Firmicutes1[col].min() > min_val_16bit):
      data_Firmicutes1[col] = data_Firmicutes1[col].astype('float16')
  elif (data_Firmicutes1[col].max() < max_val_32bit) and (data_Firmicutes1[col].min() > min_val_32bit):
    data_Firmicutes1[col] = data_Firmicutes1[col].astype('float32')



data_Firmicutes1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9328), float32(1), int16(1), int64(2), int8(2297), object(3)
memory usage: 55.3 MB


In [65]:
data_Firmicutes2 = pd.read_csv('/content/Train Dataset/Fir_train2.csv')
data_Firmicutes2.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5993 entries, 0 to 5992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9329), int64(2300), object(3)
memory usage: 534.3 MB


In [66]:
# reduce any int64 columns
int_cols = data_Firmicutes2.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Firmicutes2[col].max() < max_val_8bit) and (data_Firmicutes2[col].min() > min_val_8bit):
    data_Firmicutes2[col] = data_Firmicutes2[col].astype('int8')
  elif (data_Firmicutes2[col].max() < max_val_16bit) and (data_Firmicutes2[col].min() > min_val_16bit):
      data_Firmicutes2[col] = data_Firmicutes2[col].astype('int16')


# reduce any float64 columns
float_cols = data_Firmicutes2.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Firmicutes2[col].max() < max_val_16bit) and (data_Firmicutes2[col].min() > min_val_16bit):
      data_Firmicutes2[col] = data_Firmicutes2[col].astype('float16')
  elif (data_Firmicutes2[col].max() < max_val_32bit) and (data_Firmicutes2[col].min() > min_val_32bit):
    data_Firmicutes2[col] = data_Firmicutes2[col].astype('float32')



data_Firmicutes2.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5993 entries, 0 to 5992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9328), float32(1), int16(1), int64(2), int8(2297), object(3)
memory usage: 122.5 MB


## Fusobacteria

In [67]:
data_Fusobacteria = pd.read_csv('/content/Train Dataset/Fuso_train.csv')
data_Fusobacteria.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(8414), int64(3215), object(3)
memory usage: 16.7 MB


In [68]:
# reduce any int64 columns
int_cols = data_Fusobacteria.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Fusobacteria[col].max() < max_val_8bit) and (data_Fusobacteria[col].min() > min_val_8bit):
    data_Fusobacteria[col] = data_Fusobacteria[col].astype('int8')
  elif (data_Fusobacteria[col].max() < max_val_16bit) and (data_Fusobacteria[col].min() > min_val_16bit):
      data_Fusobacteria[col] = data_Fusobacteria[col].astype('int16')


# reduce any float64 columns
float_cols = data_Fusobacteria.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Fusobacteria[col].max() < max_val_16bit) and (data_Fusobacteria[col].min() > min_val_16bit):
      data_Fusobacteria[col] = data_Fusobacteria[col].astype('float16')
  elif (data_Fusobacteria[col].max() < max_val_32bit) and (data_Fusobacteria[col].min() > min_val_32bit):
    data_Fusobacteria[col] = data_Fusobacteria[col].astype('float32')



data_Fusobacteria.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(8413), float32(1), int16(1), int64(2), int8(3212), object(3)
memory usage: 3.7 MB


## Proteobacteria

In [69]:
data_Proteobacteria = pd.read_csv('/content/Train Dataset/Pro_train1.csv')
data_Proteobacteria.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9303), int64(2326), object(3)
memory usage: 445.9 MB


In [70]:
# reduce any int64 columns
int_cols = data_Proteobacteria.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Proteobacteria[col].max() < max_val_8bit) and (data_Proteobacteria[col].min() > min_val_8bit):
    data_Proteobacteria[col] = data_Proteobacteria[col].astype('int8')
  elif (data_Proteobacteria[col].max() < max_val_16bit) and (data_Proteobacteria[col].min() > min_val_16bit):
      data_Proteobacteria[col] = data_Proteobacteria[col].astype('int16')


# reduce any float64 columns
float_cols = data_Proteobacteria.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Proteobacteria[col].max() < max_val_16bit) and (data_Proteobacteria[col].min() > min_val_16bit):
      data_Proteobacteria[col] = data_Proteobacteria[col].astype('float16')
  elif (data_Proteobacteria[col].max() < max_val_32bit) and (data_Proteobacteria[col].min() > min_val_32bit):
    data_Proteobacteria[col] = data_Proteobacteria[col].astype('float32')



data_Proteobacteria.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9302), float32(1), int16(1), int64(2), int8(2323), object(3)
memory usage: 102.7 MB


In [71]:
data_Proteobacteria4 = pd.read_csv('/content/Train Dataset/Pro_train4.csv')
data_Proteobacteria4.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4994 entries, 0 to 4993
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9177), int64(2452), object(3)
memory usage: 445.3 MB


In [72]:
# reduce any int64 columns
int_cols = data_Proteobacteria4.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Proteobacteria4[col].max() < max_val_8bit) and (data_Proteobacteria4[col].min() > min_val_8bit):
    data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('int8')
  elif (data_Proteobacteria4[col].max() < max_val_16bit) and (data_Proteobacteria4[col].min() > min_val_16bit):
      data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('int16')


# reduce any float64 columns
float_cols = data_Proteobacteria4.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Proteobacteria4[col].max() < max_val_16bit) and (data_Proteobacteria4[col].min() > min_val_16bit):
      data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('float16')
  elif (data_Proteobacteria4[col].max() < max_val_32bit) and (data_Proteobacteria4[col].min() > min_val_32bit):
    data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('float32')



data_Proteobacteria4.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4994 entries, 0 to 4993
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9176), float32(1), int16(1), int64(2), int8(2449), object(3)
memory usage: 101.4 MB


In [73]:
data_Proteobacteria5 = pd.read_csv('/content/Train Dataset/Pro_train5.csv')
data_Proteobacteria5.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(8455), int64(3174), object(3)
memory usage: 445.0 MB


In [74]:
# reduce any int64 columns
int_cols = data_Proteobacteria5.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Proteobacteria5[col].max() < max_val_8bit) and (data_Proteobacteria5[col].min() > min_val_8bit):
    data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('int8')
  elif (data_Proteobacteria5[col].max() < max_val_16bit) and (data_Proteobacteria5[col].min() > min_val_16bit):
      data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('int16')


# reduce any float64 columns
float_cols = data_Proteobacteria5.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Proteobacteria5[col].max() < max_val_16bit) and (data_Proteobacteria5[col].min() > min_val_16bit):
      data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('float16')
  elif (data_Proteobacteria5[col].max() < max_val_32bit) and (data_Proteobacteria5[col].min() > min_val_32bit):
    data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('float32')



data_Proteobacteria5.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(8455), int16(3), int8(3171), object(3)
memory usage: 97.3 MB


## Spirochaetes

In [75]:
data_Spirochaetes = pd.read_csv('/content/Train Dataset/Spi_train.csv')
data_Spirochaetes.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1940 entries, 0 to 1939
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9144), int64(2485), object(3)
memory usage: 173.1 MB


In [76]:
# reduce any int64 columns
int_cols = data_Spirochaetes.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Spirochaetes[col].max() < max_val_8bit) and (data_Spirochaetes[col].min() > min_val_8bit):
    data_Spirochaetes[col] = data_Spirochaetes[col].astype('int8')
  elif (data_Spirochaetes[col].max() < max_val_16bit) and (data_Spirochaetes[col].min() > min_val_16bit):
      data_Spirochaetes[col] = data_Spirochaetes[col].astype('int16')


# reduce any float64 columns
float_cols = data_Spirochaetes.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Spirochaetes[col].max() < max_val_16bit) and (data_Spirochaetes[col].min() > min_val_16bit):
      data_Spirochaetes[col] = data_Spirochaetes[col].astype('float16')
  elif (data_Spirochaetes[col].max() < max_val_32bit) and (data_Spirochaetes[col].min() > min_val_32bit):
    data_Spirochaetes[col] = data_Spirochaetes[col].astype('float32')



data_Spirochaetes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1940 entries, 0 to 1939
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9143), float32(1), int16(1), int64(2), int8(2482), object(3)
memory usage: 39.4 MB


## Tenericutes

In [77]:
data_Tenericutes = pd.read_csv('/content/Train Dataset/Ten_train.csv')
data_Tenericutes.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163 entries, 0 to 2162
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9215), int64(2414), object(3)
memory usage: 192.9 MB


In [78]:
# reduce any int64 columns
int_cols = data_Tenericutes.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Tenericutes[col].max() < max_val_8bit) and (data_Tenericutes[col].min() > min_val_8bit):
    data_Tenericutes[col] = data_Tenericutes[col].astype('int8')
  elif (data_Tenericutes[col].max() < max_val_16bit) and (data_Tenericutes[col].min() > min_val_16bit):
      data_Tenericutes[col] = data_Tenericutes[col].astype('int16')


# reduce any float64 columns
float_cols = data_Tenericutes.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Tenericutes[col].max() < max_val_16bit) and (data_Tenericutes[col].min() > min_val_16bit):
      data_Tenericutes[col] = data_Tenericutes[col].astype('float16')
  elif (data_Tenericutes[col].max() < max_val_32bit) and (data_Tenericutes[col].min() > min_val_32bit):
    data_Tenericutes[col] = data_Tenericutes[col].astype('float32')



data_Tenericutes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163 entries, 0 to 2162
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9214), float32(1), int16(1), int64(2), int8(2411), object(3)
memory usage: 44.1 MB


In [ ]:
pd

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>

In [15]:
#merge dataframes rowwise
df = pd.concat([data_Actinobacteria1, data_Actinobacteria2,
                data_Bacteroidetes, data_Chlamydiae,
                data_Cyanobacteria, data_Firmicutes1,
                data_Firmicutes2, data_Fusobacteria,
                data_Proteobacteria, data_Proteobacteria4,
                data_Proteobacteria5, data_Spirochaetes,
                data_Tenericutes
                ])
df.sample(10)


,Entry,Entry name,Sequence,A,C,D,E,F,G,H,...,GO:0015168,GO:0036055,GO:0045154,GO:0003871,GO:0005388,GO:0004456,GO:0004730,GO:0052657,GO:0043771,GO:0001884
1105,P59399,HISX_LACPL,MKIINEDLASLKRLVQTKTQQLTDLKVESAVREIIANVIKNGDAAV...,0.133178,0.000000,0.070093,0.051402,0.030374,0.063084,0.011682,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2566,Q82Y50,THIC_NITEU,MKTPVSKLEKFTNDNARVDTAAVQPLPNSRKIYIQGSRADIQVPMR...,0.085443,0.012658,0.063291,0.072785,0.033228,0.072785,0.031646,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
671,P47257,Y011_MYCGE,MGKIKLKNRKALVVYDNKDDFEKNQTFALSLIKELQKKKLNAEVLL...,0.062718,0.003484,0.038328,0.059233,0.066202,0.027875,0.010453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2782,C1CFC0,RS18_STRZJ,MAQQRRGGFKRRKKVDYIAANKIEYVDYKDTELLSRFVSERGKILP...,0.088608,0.000000,0.050633,0.050633,0.037975,0.050633,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3689,B6I226,RL29_ECOSE,MKAKELREKSVEELNTELLNLLREQFNLRMQAASGQLQQSHLLKQV...,0.095238,0.000000,0.015873,0.111111,0.015873,0.031746,0.015873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4830,Q6ACQ1,GCH1_LEIXX,MTGFDRARIEAAVAEILAAVGEDPSRPGLSATPSRVADAYAEFFAG...,0.188482,0.015707,0.031414,0.099476,0.036649,0.099476,0.020942,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3050,A9R4D2,NFUA_YERPG,MITITDAAQSHFAKLLANQEEGTQIRVFVINPGTPTAECGVSYCPP...,0.073298,0.020942,0.057592,0.094241,0.031414,0.078534,0.020942,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
919,Q601J6,RL15_MYCH2,MSIRLENLSYTPGARKEKHRKGRGHAAGKGKQAGRGQSGQKKRSTV...,0.061224,0.000000,0.027211,0.068027,0.040816,0.108844,0.013605,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,B7IQV7,ATPE_BACC2,MKTFPVSIVTPDGPVYEKEVEMVSVKAESGEMGILPGHIPTVAPLK...,0.097744,0.000000,0.052632,0.082707,0.022556,0.060150,0.037594,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1055,Q4A739,RS16_MYCS5,MVKIRLKRMGSKFRPVYKIVVADARAPRDGKFIEALGHYNPSNKDF...,0.095238,0.000000,0.047619,0.035714,0.047619,0.047619,0.011905,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
Index: 47746 entries, 0 to 2162
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(11629), object(3)
memory usage: 4.2 GB


In [23]:
# free up memory
#del data_Actinobacteria1
del data_Actinobacteria2
del data_Bacteroidetes
del data_Chlamydiae
del data_Cyanobacteria
del data_Firmicutes1
del data_Firmicutes2
del data_Fusobacteria
del data_Proteobacteria
del data_Proteobacteria4
del data_Proteobacteria5
del data_Spirochaetes
del data_Tenericutes

In [35]:
# separate all columns that begin with GO
go_columns = [col for col in df.columns if col.startswith('GO')]  # binary labels
len(go_columns)

1739

In [ ]:
# remove duplicates
df2 = df.drop_duplicates()
print(len(df),len(df2))


In [ ]:
# df not needed so rm
del df

In [ ]:
# separate the target columns GO cols from the df2
y = df2[go_columns]
df3 = df2.drop(go_columns, axis=1)

In [ ]:
# df2 not needed
del df2

# There will be 3 feature sets
 * f1 contains the entire set of 9,890
 * f2 contains the set of 8,420 features that contained only compositions of amino acids, dipeptides, and tripeptides.
 * f3 contains the remaining 1,470 features that are part of f1 but not part of f2 consisting of various properties and characteristics derived from proteins



## Working with the F1

In [ ]:
f1 = df3.copy()

In [ ]:
f1.head(10)

,Entry,Entry name,Sequence,A,C,D,E,F,G,H,...,[LIV]-[LIVMGSTC]-[DET]-[RH]-[FYHCS]-x(2)-S-[GSTNP]-x-[AVC]-[FY]-[STANQ],E-[ST]-[EA]-R-E-A-[RK]-x-[LI],[EQ]-[YF]-A-[LIVM]-x(2)-[LIVM]-x(4)-[LIVMF](3)-x-G-H-x(2)-C-G,[RK]-x-[STA]-x(2)-S-x-C-Y-[SL],[GSTAP]-x(2)-[DNEQA]-[LIVM]-[GSA]-x(2)-[LIVMFYT]-[GAN]-[LIVMST]-[ST]-x(6)-R-[LIVT]-x(2)-[LIVM]-x(3)-G,[LIVMFY](2)-D-[STA]-H-x-H-[LIVMFP]-[DN],"[GA]-x(1,2)-[DE]-x-Y-x-[STAPV]-x-C-[NKR]-x-[CH]-[LIVMFYWH]",K-x-[AV]-x(4)-G-x(2)-[LIVT]-x-V-P-x(2)-[LIVC]-x(2)-[GD],"[MFY]-x-G-H-G-[LIVMC]-[GSHN]-x(3)-H-x(4)-[LIVM]-x(1,2)-[HN]-[YWVHF]",P-x(2)-R-G-[STAIV](2)-x-N-[APK]-x-[DE]
0,Q8NNB7,YBEY_CORGL,MSIEVFNESGYDGVNEEMLIDVLSFALGEMDIHPDAEASIHIVDVD...,0.107143,0.005102,0.107143,0.096939,0.035714,0.086735,0.040816,...,0,0,0,0,0,0,0,0,0,0
1,A4F9T1,YBEY_SACEN,MSIEIANESGVEVPEPSIVSVARFALDKMSVSQLAELSIVLVELDV...,0.125683,0.005464,0.081967,0.098361,0.027322,0.071038,0.038251,...,0,0,0,0,0,0,0,0,0,0
2,A4XAD4,YBEY_SALTO,MSIEIANESGADVDTDAVLAVARHALDEMGVNQLAELSVLLVDIDY...,0.146497,0.006369,0.082803,0.095541,0.012739,0.082803,0.050955,...,0,0,0,0,0,0,0,0,0,0
3,A4T2C0,YBEY_MYCGI,MSIEVSNESGIDVSESELVSVARFVIRKMDVNPAAELSMVLLDTAA...,0.089888,0.005618,0.089888,0.112360,0.028090,0.056180,0.033708,...,0,0,0,0,0,0,0,0,0,0
4,B2HLB7,YBEY_MYCMM,MSIEVSNESGIDVSETELVSVARFVIGKMDVNPGAELSMVLLDTAA...,0.095506,0.005618,0.095506,0.112360,0.028090,0.073034,0.033708,...,0,0,0,0,0,0,0,0,0,0
5,Q82B58,Y5847_STRAW,MIRFEDVSVTYDGAAEPTVRGVDFTVPEGELVLLVGPSGVGKSTVL...,0.142602,0.003565,0.053476,0.058824,0.008913,0.087344,0.021390,...,0,0,0,0,0,0,0,0,0,0
6,C1A1F6,YBEY_RHOE4,MSIEVSNESGMDISEPELISVARFVIARMDVHPAAELSMVLVDSAT...,0.088889,0.005556,0.094444,0.105556,0.027778,0.072222,0.038889,...,0,0,0,0,0,0,0,0,0,0
7,P9WKH5,YIA4_MYCTU,MSGGSSRRYPPELRERAVRMVAEIRGQHDSEWAAISEVARLLGVGC...,0.166667,0.009259,0.037037,0.101852,0.018519,0.064815,0.009259,...,0,0,0,0,0,0,0,0,0,0
8,Q9CB36,ZFPP_MYCLE,MEIIPPRLKEPLYALYELRLRQGLTASESRLPRHIAVLCDGNRRWA...,0.103053,0.015267,0.045802,0.095420,0.019084,0.080153,0.015267,...,0,0,0,0,0,0,0,0,0,0
9,P54992,YSNA_STRPR,MMQQVKRAFKYRFYPTDEQAAELSRTFGCVRLVYNKALEGRTRAWY...,0.116915,0.022388,0.039801,0.057214,0.027363,0.052239,0.012438,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(f1.columns)

9893

In [ ]:
# In the paper there are 9890 features so drop the entry, entry name and sequence column
f1.drop(['Entry','Entry name','Sequence'], axis=1,inplace=True)

In [ ]:
# convert df to numpy array and apply standard scaling
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
f1_array = scaler.fit_transform(f1)


In [ ]:
print(sys.getsizeof(f1_array))

128


In [ ]:
f1_array.shape

(9864, 9890)

In [ ]:
y.shape

(9864, 1739)

In [ ]:
# 1% neurons in the hidden layer of the input + output layer
neuron_count = int(round(((9890)+1739)*0.01,0))
neuron_count

116

In [ ]:
print(sys.getsizeof(neuron_count))

28


In [ ]:
# implement a neural network with a single hidden layer with 1 neuron
import tensorflow as tf

# our input is a (9864, 9890) array 9890 features and 9864 records
# we want to flatten it

# hidden layer has 1% neurons of the input+output and activation taken as relu

# output layer has 1739 neurons as there are 1739 binary labels

m1 = tf.keras.Sequential(
     [
    tf.keras.layers.Dense(units=neuron_count, activation=tf.keras.activations.relu, kernel_initializer=tf.keras.initializers.HeUniform(), input_dim=9890), # hidden layer with 1 neuron
    tf.keras.layers.Dense(units=1739, activation=tf.keras.activations.sigmoid)  # output layer with 1739 neurons
])




/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
print(sys.getsizeof(m1))

56


## Metrics Used in the study
1. Precision
2. Recall
3. F1 Score
4. Micro Average
5. Macro Average
6. Weighted Average
7. Samples Average
8. Zero One Loss
9. Hamming Loss
10. Matthew's Correlaton Coefficent
11. Consolidated Performance Metric

In [ ]:
# implementing custom functions
import keras.backend as K

def Custom_Hamming_Loss(y_true, y_pred):
  return K.mean(y_true*(1-y_pred)+(1-y_true)*y_pred)

def Custom_Matthews_Correlation(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())




In [ ]:
m1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics= [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.F1Score(), Custom_Hamming_Loss, Custom_Matthews_Correlation]
    )


In [ ]:
m1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 116)                 │       1,147,356 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1739)                │         203,463 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,350,819 (5.15 MB)

 Trainable params: 1,350,819 (5.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
m1.fit(f1_array,
       y,
       epochs=100,
       batch_size=50
       )

Epoch 1/100


AttributeError: module 'keras.backend' has no attribute 'mean'

In [ ]:
m1.save("model.keras")

In [ ]:
import os



print(f"Model file size: {os.path.getsize('/content/model.keras') / (1024 * 1024):.2f} MB")


Model file size: 15.48 MB


In [ ]:
m1

<Sequential name=sequential, built=True>